<a href="https://colab.research.google.com/github/hail-members/distributed-deep-learning/blob/main/JAX_FLAX/2.JAX2JIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JAX의 JIT 컴파일

jax가 순수함수를 그렇게 고집하는건 JIT (just-in-time) 컴파일을 하기 위해서입니다. JIT 컴파일은 함수를 실행하기 전에 최적화된 기계어 코드로 변환하여 실행 속도를 크게 향상시키는 역할을 합니다. 그냥 python은 인터프리터 언어이기 때문에... 그 라인에 도착해서 하나씩 컴파일 하고 실행하고 잊는... 방식을 반복하지만

JAX는 순수함수만을 대상으로 JIT 컴파일을 수행할 수 있기 때문에, 함수가 외부 상태에 의존하지 않고 입력에만 의존하도록 설계해야 합니다.

함수를 쓰는 방식부터 - jit 데코레이터를 쓰는 방식(더 일반적인 방식)으로 봅시다.
jaxpr 은 jax expression 이라는 말로 컴파일된 형태를 의미합니다. 이렇게 변경시켜주는 것을 jax.make_jaxpr()


In [1]:
import jax
import jax.numpy as jnp
from jax import make_jaxpr

def my_function(x, y):
  return x * y + 2

print(make_jaxpr(my_function)(3., 4.))

{ lambda ; a:f32[] b:f32[]. let
    c:f32[] = mul a b
    d:f32[] = add c 2.0:f32[]
  in (d,) }


이거 보면 이런식으로 기계어 같아보이는 뭔가가 나옵니다. 이 상태로 컴파일된걸 그냥 쓰는겁니다 람다 함수기 때문에 a,b를 입력으로 넣어주면 그 안에서 연산을 정해진대로 수행합니다.

이건 make_jaxpr 을 한 시점에서 결정된겁니다. jaxpr로 결정되면 my_function 을 변경시켜도, make_jaxpr 로 만들어진 jaxpr이 변경되는게 아닙니다.

이번엔 순수함수가 아닌 경우 어떻게 되는지 봅시다

In [2]:
def log2_with_print(x):
    print("printed x:", x)
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2


print(jax.make_jaxpr(log2_with_print)(3.))

printed x: JitTracer<~float32[]>
{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0:f32[]
    d:f32[] = div b c
  in (d,) }


이전에 말했던것처럼 print 는 외부와 상호작용 하는 것이기 때문에 순수함수가 아니게됩니다. 보면 여기서 printed x: 라고 되어있는 부분은 {} 안에 들어가 있지 않습니다. 이건 jaxpr에 포함되지 않았다는 뜻입니다. 이렇게 외부로 왔다 갔다 하는 값은 jaxpr에 포함되지 않아서 컴파일이 통하지 않습니다. 그래서 굳이 jax를 썼음에도 함수가 속도가 빨리지지 않는다는 문제가 발생합니다.

다음과 같은 함수에서는 어떻게 되는지 확인해볼까요?

In [3]:
def square_if_gt_2(x):
    if x.ndim > 2:
        return x**2
    else:
        return x


print(make_jaxpr(square_if_gt_2)(jax.numpy.array([1, 2, 3])))

{ lambda ; a:i32[3]. let  in (a,) }


이전에 설명했던대로 이렇게 조건문이 있는 경우에 정해진 수대로 돌아가는게 아니라 한번 실행됐을때 입력값에 따라서 컴파일이 됩니다.

그래서 여기는 3개짜리 어레이가 들어가니까 jaxpr에서도 length 3개짜리 어레이로 컴파일된걸 볼 수 있습니다. 만약에 5개짜리 어레이를 넣으면 어떻게 될까요?

In [4]:
print(make_jaxpr(square_if_gt_2)(jax.numpy.array([1, 2, 3, 4, 5])))

{ lambda ; a:i32[5]. let  in (a,) }


## JIT으로 컴파일하기

make_jaxpr 로 jaxpr 을 얻는 것을 봤습니다만, 이는 실제 컴파일을 하는게 아닌 컴파일 됐을때 표현식이 어떻게 되는지를 보여주는 것입니다. 실제로 컴파일을 해서 속도를 올리기 위해서는 JIT 컴파일을 사용해야 합니다.

이는 jax.jit 라는 메서드로 구현되어있으며 일반적으로 데코레이터로 사용됩니다. 이걸로 감싸진 함수는 처음 실행될때 jaxpr로 컴파일되고, 이후에는 그 컴파일된 코드를 재사용하게 됩니다.

예를 들어 selu라는 활성함수에 대해서 시간을 측정해보고, jit 컴파일을 적용한 경우를 비교해봅시다.

In [5]:
def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)


x = jnp.arange(1000000)
%timeit selu(x).block_until_ready()


30.9 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
selu_jit = jax.jit(selu)

# Warm up
selu_jit(x).block_until_ready()

%timeit selu_jit(x).block_until_ready()

3.01 ms ± 805 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


참고로 여기서 %timeit 은 주피터 노트북에서 제공하는 매직커맨드로, 해당 셀의 실행 시간을 측정해주는 것이고 block_until_ready() 는 jax에서 비동기적으로 실행되는 연산이 완료될 때까지 기다리는 함수입니다. torch 에서 cuda sync 맞춰주는걸로 시간 측정한다고 말했던거 기억하시나요? 여기서도 jax가 비동기적으로 연산을 수행하기 때문에, 정확한 시간을 측정하기 위해서는 block_until_ready() 를 호출하여 연산이 완료될 때까지 기다려야 합니다.

참고로, jit 의 경우는 selu_jit 을 한번 실행시키고 나서 시간을 측정합니다. 이는 처음 호출될때야 비로소 컴파일을 하기 때문에 그렇습니다. 첫 호출에서 jaxpr 을 만들고 컴파일을 수행한 후, 이후 호출부터는 이미 컴파일된 코드를 사용하게 됩니다. 아까 위에서 보여드린대로 그래서 첫 호출에서 어떤 값을 넣느냐에 따라 컴파일된 코드가 달라질 수 있습니다. (예: length 3짜리 어레이 넣었을때랑 length 5짜리 어레이 넣었을때랑 다름)

## 순수함수가 아닌 경우 JIT 컴파일

이번에는 순수함수가 아닌 경우에 jit 컴파일을 한번 시도해봅시다.

In [7]:
def f(x):
    if x > 0:
        return x
    else:
        return 2 * x


f_jit = jax.jit(f)
f_jit(10) # 에러 발생.

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function f at /tmp/ipython-input-2545734610.py:1 for jit. This concrete value was not available in Python because it depends on the value of the argument x.
See https://docs.jax.dev/en/latest/errors.html#jax.errors.TracerBoolConversionError

In [8]:
# 입력 n이 조건에 포함된 while 반복문.

def g(x, n):
    i = 0
    while i < n:
        i += 1
    return x + i


g_jit = jax.jit(g)
g_jit(10, 20) # 에러 발생.

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function g at /tmp/ipython-input-1346608175.py:3 for jit. This concrete value was not available in Python because it depends on the value of the argument n.
See https://docs.jax.dev/en/latest/errors.html#jax.errors.TracerBoolConversionError

위와 같은 예시들은 입력값에 따라서 연산이 달라지기 떄문에 컴파일된 표현식이 달라지게 된다는 문제점이 있어서 제대로 돌아가지 않습니다.

컴퓨터구조 와 같은 수업에서 C언어 로 작성된 코드를 컴파일하면 어떤 instructions들로 나오는지 보는 것들을 해본 적이 있을 텐데 (아님 미안) 여기도 비슷합니다.

물론 c언어에서는 bne라던가 beq 같은 분기 명령어가 있고 jax.lax.cond 라는 걸 이용해서 조건문을 활용한 코딩을 할 수도 있습니다만, 일단 머리가 아프고 더 쉽게 구현하는 방법이 선호됩니다.

### 방법 1

In [9]:
@jax.jit
def loop_body(prev_i):
    return prev_i + 1


def g_inner_jitted(x, n):
    i = 0
    while i < n:
        i = loop_body(i)
    return x + i


g_inner_jitted(10, 20)


Array(30, dtype=int32, weak_type=True)

그 방법은 위와같이 wrapper 르 씌워서 jit을 적용하는 부분을 반복문의 내부 (또는 조건에 의해서 영양 받지 않는 부분)으로 제한하는겁니다.

### 방법 2

In [10]:
f_jit_correct = jax.jit(f, static_argnums=0)
print(f_jit_correct(10))

g_jit_correct = jax.jit(g, static_argnames=['n'])
print(g_jit_correct(10, 20))

10
30


또 다른 방법은 jax.jit 할때 static_argnums 또는 static_argnames 라는 옵션을 주는 것입니다. 이 옵션들은 해당 인자들이 정적으로 고정되어 있다고 jax에게 알려주는 역할을 합니다. 즉, 이 인자들은 컴파일 시점에 고정된 값으로 간주한 상태로 컴파일된 코드를 씁니다.

그럼 조건, 즉 입력 인자가 달라지면 컴파일 된 코드가 틀리지 않냐고요? 맞습니다. 그래서 이 방법은 인자가 자주 바뀌지 않는 경우에만 씁니다. 만약 바뀌더라도, jax는 이렇게 static_argnums/static_argnames 으로 선언된 것은 나중에 변경된 것을 간단하게 리컴파일 할 수 있게 지원합니다.

앞서 말씀드렸듯 jax.jit 을 데코레이터를 써서 표현하자면 다음과 같습니다

In [11]:
from functools import partial

@partial(jax.jit, static_argnames=['n'])
def g_jit_decorated(x, n):
    i = 0
    while i < n:
        i += 1
    return x + i


print(g_jit_decorated(10, 20))


30


보시다 시피 이제 partial 을 써서 jax.jit 으로 감싸되, static_argnames 라는 옵션을 주었습니다. 이 옵션은 n 이라는 인자가 정적으로 고정되어 있다고 jax에게 알려주고, partial을 통해서 그 인자가 없이 호출될 수 있도록 합니다. 이제 g_jit_correct 는 n 이라는 인자를 받지 않고도 호출될 수 있습니다.

그럼 static_argnums/static_argnames 옵션을 사용하면 이렇게 조건문이 달린 함수를 자유롭게 써도 될까요?

## JIt 컴파일과 캐싱의 주의점

jax.jit 은 컴파일된 코드를 캐싱합니다. 즉, 동일한 입력 형태에 대해 이미 컴파일된 코드가 있다면, 이를 재사용합니다. 이게 성능향상에 도움이 되기 때문에 jax를 쓴다는건 이렇게 컴파일하고 안건드리겠다는 선언이나 다름없습니다.

만약 static_argnums/static_argnames 옵션을 사용하여 특정 인자를 정적으로 고정시켰다면, 해당 인자가 변경될 때마다 jax는 새로운 컴파일된 코드를 생성합니다. 예를 들어, g_jit_correct(10, 20) 를 호출할때 jax는 n=20 이라고 고정한 상태의 g_jit_correct 를 컴파일합니다. 만약 이후에 g_jit_correct(10, 30) 을 호출하면, jax는 n=30 으로 고정된 새로운 버전을 컴파일해야합니다. 이 컴파일은 시간이 오래걸리니까 이렇게 반복해서 리컴파일할때는 jit을 쓰지 않는게 좋습니다.

다시말해서 어떤 방식으로든 (static_argnums/static_argnames 인자를 갖고있는 함수라던가 아니면 jax.jit 을 아예 명시적으로 for문 내에 넣는다던가) jax.jit 이 컴파일을 하는 일을 만들지 않는게 좋으며, 특히 반복문 내에서 연속적으로 부르는걸 삼가야 합니다.

In [12]:
def unjitted_loop_body(prev_i):
    return prev_i + 1


def g_inner_jitted_partial(x, n):
    i = 0
    while i < n:
    # 하지마세요!
    # 매번 partial이 다른 해쉬의 함수를 반환합니다.
        i = jax.jit(partial(unjitted_loop_body))(i)
    return x + i


def g_inner_jitted_lambda(x, n):
    i = 0
    while i < n:
    # 하지마세요!
    # lambda 또한 매번 다른 해쉬의 함수를 반환합니다.
        i = jax.jit(lambda x: unjitted_loop_body(x))(i)
    return x + i


def g_inner_jitted_normal(x, n):
    i = 0
    while i < n:
    # 이건 괜찮습니다!
    # JAX가 캐싱되고 컴파일된 함수를 다시 찾을 수 있습니다.
        i = jax.jit(unjitted_loop_body)(i)
    return x + i


print("jit called in a loop with partials:")
%timeit g_inner_jitted_partial(10, 20).block_until_ready()


print("jit called in a loop with lambdas:")
%timeit g_inner_jitted_lambda(10, 20).block_until_ready()


print("jit called in a loop with caching:")
%timeit g_inner_jitted_normal(10, 20).block_until_ready()


jit called in a loop with partials:
453 ms ± 18.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
jit called in a loop with lambdas:
437 ms ± 9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
jit called in a loop with caching:
2.63 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


마지막 녀석을 제외하고는 매번 컴파일 하는 것이기 때문에(캐싱되지 않기 때문에) 시간이 훨씬 오래걸립니다.

# 그 외의 알아둬야하는 기능들

## 자동 벡터화

jax는 선형대수적 연산을 가속하는게 목표입니다. 그래서 벡터화(vectorization) 라는 기능을 제공합니다. 벡터화는 스칼라 연산을 벡터나 행렬 연산으로 변환하여 한 번에 여러 데이터를 처리하는 것을 의미합니다. jax에서는 jax.vmap 이라는 함수를 통해 벡터화를 지원합니다.

먼저 자동 벡터화 이전에 수동 벡터화를 하는 방법에 대해서 생각해봅시다.

1차원 컨볼루션을 한다고 생각해봅시다. 원래 연산은 다음과 같습니다.

In [13]:
import jax
import jax.numpy as jnp

x = jnp.arange(5)
w = jnp.array([2., 3., 4.])

def convolve(x, w):
    output = []
    for i in range(1, len(x)-1):
        output.append(jnp.dot(x[i-1:i+2], w))
    return jnp.array(output)

convolve(x, w)

Array([11., 20., 29.], dtype=float32)

In [14]:
xs = jnp.stack([x, x])
ws = jnp.stack([w, w])

In [15]:
def manually_batched_convolve(xs, ws):
    output = []
    for i in range(xs.shape[0]):
        output.append(convolve(xs[i], ws[i]))
    return jnp.stack(output)


manually_batched_convolve(xs, ws)


Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

이거는 그냥 결과는 정상적으로 나오는데 결국 연산은 jax를 사용하지 않기 때문에 속도가 느립니다. 이걸 수동 벡터화로 바꿔봅시다.

In [16]:
def manually_vectorized_convolve(xs, ws):
    output = []

    for i in range(1, xs.shape[-1] -1):
        output.append(jnp.sum(xs[:, i-1:i+2] * ws, axis=1))
    return jnp.stack(output, axis=1)


%timeit manually_vectorized_convolve(xs, ws).block_until_ready()


856 µs ± 131 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


별거 없습니다. 그냥 convolve 함수에서 for 문으로 돌아가던걸 jnp.sum 을통해 직접 벡터연산으로 선언해주는겁니다.

In [17]:
auto_batch_convolve = jax.vmap(convolve)

auto_batch_convolve(xs, ws)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

물론 수동으로 하면 머리아프니까.. 이렇게 수동으로 바꾸는게 아니라 jax.vmap 을 이용해서 자동으로 벡터화를 할 수도 있습니다.

물론 어느 차원에 대해서 벡터화할지를 지정할 수 있습니다. 예를들어 배치차원에대해서 벡터화하는게 일반적이니까 transpose 한 데이터에대해서 vector화 할 차원을 지정하고 jax.vmap 을 적용하는 것을 생각해봅시다.

In [18]:
auto_batch_convolve_v2 = jax.vmap(convolve, in_axes=1, out_axes=1)

xst = jnp.transpose(xs)
wst = jnp.transpose(ws)

auto_batch_convolve_v2(xst, wst)

Array([[11., 11.],
       [20., 20.],
       [29., 29.]], dtype=float32)

## 그레디언트 중지

torch 에서 .detach() 쓰는거 아시나요? jax에서는 jax.lax.stop_gradient를 이용해서 흘려줄 부분과 아닌 부분을 나눕니다.

예를 들어 강화학습에서 TD learning 을 한다면

$PE=V_\theta(s') + r - V_\theta(s)$

이고 이때 $V_\theta(s)$ 는 그레디언트를 흘리고 $V_\theta(s')$ 는 흘리지 않습니다. 타겟을 정해놓고 그 타겟에 맞춰서 현재 값을 조정하는거니까요. 이럴때 jax.lax.stop_gradient 를 이용해서 $V_\theta(s')$ 에 대해서는 그레디언트를 흘리지 않도록 할 수 있습니다.

```python
def td_loss(theta, s_tm1, r_t, s_t):
    v_tm1 = value_fn(theta, s_tm1)
    target = r_t + value_fn(theta, s_t)
    return (target - v_tm1) ** 2


td_update = jax.grad(td_loss)
delta_theta = td_update(theta, s_tm1, r_t, s_t)
```

```python
def td_loss_with_stop_gradient(theta, s_tm1, r_t, s_t):
    v_tm1 = value_fn(theta, s_tm1)
    target = r_t + value_fn(theta, s_t)
    return (jax.lax.stop_gradient(target) - v_tm1) ** 2


td_update = jax.grad(td_loss_with_stop_gradient)
delta_theta = td_update(theta, s_tm1, r_t, s_t)
```


## JAX의 난수

jax의 난수에는 고려할 사항이 있습니다. 일단 알아야 할 것은 numpy 와는 다른 방식으로 난수를 생성한다는 점, 그리고 함수형으로 구현하고 병렬화하기에 이 numpy 방식의 난수생성을 그대로 갖다가 쓰기 어렵다는 점 입니다. 그래서 다른 방식으로 난수를 생성합니다.

### numpy 의 난수

아시다시피 numpy 는 seed를 통해서 난수를 생성하는 시드를 고정하고 거기서부터 난수를 생성합니다. 예를 들어 아래의 코드는 몇백번을 돌려도 동일한 값을 출력합니다.

In [19]:
import numpy as np
np.random.seed(0)
np.random.rand()

0.5488135039273248

근데 한번 생각해보세요. 이걸 병렬 처리로 돌리게 되면 각각의 쓰레드에 열린 프로그램이 서로 다른 시드를 가져야만 합니다. 근데 한번 더 생각해보면 순수함수를 쓰는 jax에서 병렬화된 각 프로그램마다 다른 입력값을 넣어줄수 있을리가 없습니다. 아까본 jaxpr같이 컴파일이 되어야하는데 입력값이 결정되어서 실행하면 버튼누르듯 실행되어야하는데 서로다른 seed를 갖고 있다는것은 입력값이 다르다는 것이고 그럼 동일하게 컴파일 된 하나의 프로그램을 병렬하는게 아니라 서로다른 시드를 갖고 있는 서로다른 컴파일된 코드를 돌리는겁니다.

seed가 다른걸 각각 컴파일 해서 돌려야한다는건.. 결국 오히려 컴파일 하면서 시간이 오래 걸린다는 뜻입니다. jax는 이걸 해결하기 위해서 자체적인 난수 생성 방식을 사용합니다.

이때는 key라는걸 사용합니다. seed 와 비슷한데, seed는 단일 정수로 지정되어 이 시드가 있으면 난수 배열이 고정된다면, key 는 그 상위 개념으로 난수를 생성하는 시드를 여러개를 담고 있는 객체라고 생각하면됩니다.

In [20]:
from jax import random
key = random.PRNGKey(42)
print(key)

print(random.normal(key))
print(random.normal(key))

[ 0 42]
-0.028304616
-0.028304616


이러면 별 차이 없어보이지만 더 중요한기능은 key 를 쪼개는 기능입니다. jax.random.split 이라는 함수를 통해서 key 를 여러개로 쪼갤 수 있습니다. 왜 쪼개는게 중요하냐? 병렬화 할때 같은 컴파일 코드들이 서로다른 random key 를 가지고 서로다른 난수를 생성할 수 있기 때문입니다.



In [21]:
print("old key", key)
new_key, subkey = random.split(key)
del key  # 오래된 키는 지워버리며 나중에라도 사용하지 않습니다..
normal_sample = random.normal(subkey)
print(r"    \---SPLIT --> new key   ", new_key)
print(r"             \--> new subkey", subkey, "--> normal", normal_sample)

print(random.normal(subkey))
del subkey  # 서브키도 사용후에 제거해야 합니다.
key = new_key  # 만약에 한번 이 키를 다시 생성해야 한다면 new_key는 키로 사용됩니다.

print(random.normal(key))

old key [ 0 42]
    \---SPLIT --> new key    [1832780943  270669613]
             \--> new subkey [  64467757 2916123636] --> normal 0.60576403
0.60576403
0.07592554


## pytree 와 jax의 mapping

jax 에서는 pytree 라는 개념을 사용합니다. pytree 는 파이썬의 기본 자료구조 (리스트, 튜플, 딕셔너리 등) 와 사용자 정의 객체를 포함하여 다양한 형태의 데이터를 트리 구조로 표현하는 방법입니다. 이 구조를 크게 이해할 필요는 없지만, 대충 jax는 함수형 언어기 때문에 이 변수를 뭉쳐서 넣기 좋은 컨테이너가 필요해서 만들었다 - 정도로 이해하고 계시면됩니다.

중요한건 이 pytree 에서 mapping 이라는 기능입니다. tree_map 이라는 함수를 통해서 pytree 의 각 요소에 대해서 함수를 적용할 수 있습니다. 예를 들어, pytree에 신경망의 parameter 들이 들어있다면 이를 초기화 하기 위해서 random number 를 넣어줘야하는데 그걸 tree_map 을 통해서 쉽게 할 수 있습니다.

In [22]:
import jax
import jax.numpy as jnp


example_trees = [
    [1, 'a', object()],
    (1, (2, 3), ()),
    [1, {'k1': 2, 'k2': (3, 4)}, 5],
    {'a': 2, 'b': (2, 3)},
    jnp.array([1, 2, 3]),
]

for pytree in example_trees:
  leaves = jax.tree_util.tree_leaves(pytree)
  print(f"{repr(pytree):<45} has {len(leaves)} leaves: {leaves}")


[1, 'a', <object object at 0x7cd86dd84650>]   has 3 leaves: [1, 'a', <object object at 0x7cd86dd84650>]
(1, (2, 3), ())                               has 3 leaves: [1, 2, 3]
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 5 leaves: [1, 2, 3, 4, 5]
{'a': 2, 'b': (2, 3)}                         has 3 leaves: [2, 2, 3]
Array([1, 2, 3], dtype=int32)                 has 1 leaves: [Array([1, 2, 3], dtype=int32)]


선언 자체를 pytree라는 특정한 클래스로 만들어서 선언하는게 아니라, 위 처럼 리스트, 튜플, 딕셔너리 등 파이썬의 기본 자료구조를 사용해서 선언하면 그걸 하나 하나 열면서 끝까지 들어가면 leaf에 해당하는 부분에 함수를 적용해야하는 구나 ~ 라는 식으로 tree_map 을 쓸 수 있습니다. 좀 더 쉬운 예제를 보자면 다음과 같습니다

In [23]:
list_of_lists = [
    [1, 2, 3],
    [1, 2],
    [1, 2, 3, 4]
]


jax.tree_util.tree_map(lambda x: x*2, list_of_lists)


[[2, 4, 6], [2, 4], [2, 4, 6, 8]]

위 예시는 제곱하는 함수를 list of lists라는 pytree 에 적용하는 예시입니다. leaf에 적용된 걸 볼수 있습니다.

아까 얘기한것처럼 신경망의 파라미터에 대해서 랜덤 초기화를 할때도 이렇게 쓸 수 있습니다.

In [24]:
import jax.numpy as jnp
from jax import random

def init_mlp_params(layer_widths, key):
    params = []
    for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
        params.append(
            dict(weights=random.normal(key, shape=(n_in, n_out)) * jnp.sqrt(2/n_in), biases=jnp.ones(shape=(n_out,))
            )
       )
    return params


key = random.PRNGKey(42)
params = init_mlp_params([1, 128, 128, 1], key)

print(params)

jax.tree_util.tree_map(lambda x: x.shape, params)


[{'weights': Array([[-0.04002877,  0.6606242 ,  0.41818714,  0.21714671, -0.17540888,
         0.30677566, -2.0377107 ,  1.0689473 ,  0.73738456,  1.2871753 ,
        -0.5437603 ,  1.6119536 ,  2.0446506 ,  1.5286328 , -0.07961062,
         1.2863609 ,  0.7882065 ,  0.30979365, -2.0485008 ,  1.0807244 ,
        -0.341599  , -1.6678966 , -2.7420447 ,  0.50383425, -0.3409947 ,
         1.7184497 , -1.9731419 , -0.7562774 ,  0.38279307,  2.1780643 ,
         0.98078346, -0.14690392, -0.71036935,  0.9576821 ,  0.15676567,
        -0.49179202,  0.6433298 ,  0.322212  , -0.7877809 , -1.2487663 ,
        -0.3019355 ,  0.43559375, -0.2647677 ,  0.13242048,  0.52874786,
        -1.4951242 ,  0.6316881 ,  1.7123226 ,  0.6136047 , -0.99527884,
         0.24945721, -0.28100944, -0.30844915,  1.8176203 ,  0.5308272 ,
        -0.25183904, -0.3391156 , -0.5795661 ,  0.5191829 ,  1.6799393 ,
        -1.4686499 , -1.1233196 ,  1.4970272 , -0.51215523, -0.07794855,
        -2.9027944 ,  2.1227539 , -2.0

[{'biases': (128,), 'weights': (1, 128)},
 {'biases': (128,), 'weights': (128, 128)},
 {'biases': (1,), 'weights': (128, 1)}]

## JAX의 짱쉬운 병렬처리

진짜 쉽습니다. 그냥 pmap 이라는 함수를 쓰면 됩니다. 아까 선언했던 convolve를 써봅시다. vmap 했던것과 똑같이 적용하면됩니다.

In [25]:
import numpy as np

x = np.arange(5)
w = np.array([2., 3., 4.])

def convolve(x, w):
  output = []
  for i in range(1, len(x)-1):
    output.append(jnp.dot(x[i-1:i+2], w))
  return jnp.array(output)

convolve(x, w)



Array([11., 20., 29.], dtype=float32)

In [26]:
n_devices = jax.local_device_count()

print(n_devices)

xs = np.arange(5 * n_devices).reshape(-1, 5)
ws = np.stack([w] * n_devices)

xs

1


array([[0, 1, 2, 3, 4]])

In [27]:
jax.vmap(convolve)(xs, ws)

Array([[11., 20., 29.]], dtype=float32)

In [28]:
import time
start = time.time()
jax.vmap(convolve)(xs, ws)
print(f"vmap : {time.time()-start:.4f} sec")
start = time.time()
jax.pmap(convolve)(xs, ws)
print(f"pmap : {time.time()-start:.4f} sec")

vmap : 0.0088 sec
pmap : 0.0596 sec


위의 실행시간은 디바이스 갯수에 따라서 성능에 차이가 있습니다~